In [4]:
import tensorflow as tf
import PIL
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np

## 5-1. 5차 학습 전 평가


### 4차 학습 모델 

- 기존 학습 방법들중 가장 성능이 좋은 모델 사용
- VGG16보다 단순한 ResNet50을 전이학습을 진행하여 구현한 모델


### 2차 데이터셋

- kill 이미지 1261
- non-kill 이미지 932

현재 데이터가 학습과 평가 데이터가 구분되어있지 않습니다.

### 5차 학습 전 평가 특징

- 2차 데이터셋에서 ROI를 추출하여 평가 데이터로 사용합니다. 
- 2차 데이터셋 모두 사용하여 현재의 모델을 평가합니다. 추가된 데이터셋에 대한 정확도가 충분히 높다면 2차 데이터셋에 대한 학습이 필요하지 않기 때문입니다. 

In [5]:
ADD_IMAGE_PATH = './dataset/kill_log_set_2/'

In [6]:
add_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ADD_IMAGE_PATH,
    label_mode = 'binary',
    image_size = (540, 960),
)

Found 2193 files belonging to 2 classes.


In [7]:
# 대충 본다.
def crop_roi(image, label): 
    
    cropped_image = tf.image.crop_to_bounding_box(
        image, 10, 200, 190, 560
    )
    #resized_image = tf.image.resize(cropped_image, [190, 560], method = 'lanczos3')
    #casted_image = resized_image / 255
    return cropped_image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# ROI의 과정은 Data Augmentation의 수행이 아니므로 학습 데이터, 검증 데이터 모두 적용해야합니다.
def make_crop_dataset(dataset):
    dataset = dataset.cache()
    dataset = dataset.map(crop_roi, num_parallel_calls=AUTOTUNE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [10]:
add_cropped_dataset = make_crop_dataset(add_train_dataset)

In [83]:
func_saved_model_t4 = tf.keras.models.load_model('./kill_log_models/model_t4.hdf5')

In [78]:
func_saved_model_t4.evaluate(add_cropped_dataset)

69/69 [==============================] - 22s 311ms/step - loss: 0.1219 - accuracy: 0.9676


[0.1219450831413269, 0.9676242470741272]

### 5차 학습전 평가의 결과


- 평가 데이터의 정확도는 96.7%

ROI를 추출하고, 추출된 이미지를 4차 학습 모델에 대해 평가한 결과, 기존의 98.7%에서 약 2%가 감소한 것을 확인할 수 있습니다. 2%의 감소는 2차 데이터셋에 대한 학습이 충분히 유의미할 수 있을 것으로 판단됩니다.

- 랜덤하게 추출하여 2차 데이터셋을 2차 데이터셋(학습용), 2차 테스트셋으로 분류합니다.

## 5-2. 5차 학습 


### ResNet

- 4차 학습 모델 사용
- VGG16보다 단순한 ResNet50사용


### 1차 데이터셋 (기존 학습)

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 5차 학습 특징

- ROI를 수행합니다.
- 2차 테스트 셋으로 4차 학습 모델을 평가합니다.
- 그 후, 4차 학습 모델을 2차 데이터 셋으로 추가학습을 진행합니다.

In [11]:
ADD_TRAIN_PATH = './cropped_dataset/kill_log_dataset_2'
ADD_TEST_PATH = './cropped_dataset/kill_log_testset_2'

In [ ]:
# 데이터셋 준비하기

In [90]:
# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장. (모델을 저장하는 것은 ) 
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t5.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [15]:
# 모델 로드하기

In [ ]:
# 모델 추가학습 하기

In [97]:
# 모델 평가하기

14/14 [==============================] - 2s 125ms/step - loss: 0.0880 - accuracy: 0.9772


[0.08803118765354156, 0.9771689772605896]

In [99]:
# 가장 좋은 모델 로드하기

In [100]:
# 가장 좋은 모델 평가하기

14/14 [==============================] - 2s 111ms/step - loss: 0.0880 - accuracy: 0.9772


[0.08803118765354156, 0.9771689772605896]

## 5-3. 1차 테스트 셋에 대한 평가

In [13]:
# 1차 테스트셋 로드하기

Found 310 files belonging to 2 classes.


In [101]:
# 1차 테스트셋 평가하기

10/10 [==============================] - 1s 110ms/step - loss: 0.0106 - accuracy: 1.0000


[0.01064381469041109, 1.0]

### 5차 학습 결과

- 1차 평가 데이터의 정확도는 100%
- 2차 평가 데이터의 정확도는 97.7%

2차 평가 데이터에 대해서도 추가 학습 모델의 정확도가 매우 높은 것을 확인할 수 있습니다. 이전 학습에서 Data Augmentation을 통한 정확도 개선을 확인하였으므로 현재의 학습 방법에 Data Augmentation까지 추가하여 학습을 진행해보겠습니다.

## 6-1. 6차 학습


### ResNet

- 4차 학습 모델 사용
- VGG16보다 단순한 ResNet50사용


### 1차 데이터셋 (기존 학습)

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 6차 학습 특징


- 4차 학습 모델을 2차 데이터 셋으로 추가학습을 진행합니다.
- ROI을 추출한 뒤 Data Augmentation을 통해 데이터셋의 크기를 늘립니다.

In [15]:
# Data Augmentation 구현 들고오기

In [16]:
# Data Augmentation 적용

# validation, test set은 그대로 사용

NameError: name 'make_augmented_dataset' is not defined

In [115]:
# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장. (모델을 저장하는 것은 ) 
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t6.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [17]:
# 모델 로드하기

In [ ]:
# 모델 추가학습하기

In [117]:
# 모델 평가하기

14/14 [==============================] - 2s 110ms/step - loss: 0.0958 - accuracy: 0.9703


[0.09581686556339264, 0.9703196287155151]

In [19]:
# 가장 좋은 모델 로드하기

In [ ]:
# 가장 좋은 모델 평가하기

In [119]:
# 가장 좋은 모델 1차 데이터로 평가하기

10/10 [==============================] - 1s 108ms/step - loss: 0.0106 - accuracy: 1.0000


[0.01064381469041109, 1.0]

### 6차 학습의 결과

- 학습 데이터의 가장 높은 정확도는 99%

- 1차 평가 데이터의 정확도는 100%

- 2차 평가 데이터의 정확도는 97.7%

ROI 추출 및 Data Augmentation까지 추가하여 학습을 진행해보았으나, 5차 학습과의 정확도 차이가 발생하지 않았습니다. 이로 미루어볼 때, 2차 학습 데이터를 추가학습한 5차 모델은 Data Augmentation를 통해 생성되는 새로운 이미지들에 대해서도 충분히 예측가능하다는 것으로 생각해볼 수 있습니다. 여러가지 학습 방법으로 학습을 진행해보았습니다. 가장 좋은 모델인 6차 학습 모델에 대해서 파인튜닝을 통해 정확도를 향상시키도록 하겠습니다.

## 6-2. 6차 학습에 대한 파인튜닝

평가 데이터의 정확도가 충분히 높기 때문에 파인튜닝을 통해서 현재 모델을 고도화하는 방법을 선택할 수 있습니다.


### ResNet

- 6차 학습 모델 사용
- VGG16보다 단순한 ResNet50사용


### 1차 데이터셋 

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 6차 파인튜닝 특징


- 전이학습을 위해 가중치 갱신이 불가능하도록 설정한 것을 파인튜닝을 위해 가능하도록 설정합니다.
- 1, 2차의 데이터셋을 결합합니다.
- ROI을 추출한 뒤 Data Augmentation을 통해 데이터셋의 크기를 늘립니다.

In [18]:
# 6차 모델 로드하기

In [20]:
# 6차 모델 학습 가능하도록 변경하기

### 데이터셋 구성하기

- 1차 데이터셋 로드하기

In [19]:
# 1차 데이터셋 로드하기

Found 2618 files belonging to 2 classes.
Using 2357 files for training.
Found 2618 files belonging to 2 classes.
Using 261 files for validation.


In [22]:
# 1, 2차 데이터셋이 결합된 데이터셋


In [23]:
# 1, 2차 데이터셋이 결합된 데이터셋


In [24]:
# 결합된 데이터셋에 Data Augmentation 추가


In [29]:
# val_loss에 따른 빠른 종료 -> 빠른 구현을 위해서 선택적으로 적용할 수 있다.
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장. (모델을 저장하는 것은 ) 
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t62.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [30]:
# 파인튜닝을 위한 학습루프 추가

In [31]:
# 파인튜닝을 위한 전체 데이터 학습

Epoch 1/20
129/129 [==============================] - 102s 478ms/step - loss: 0.0883 - accuracy: 0.9738 - val_loss: 0.0502 - val_accuracy: 0.9862

Epoch 00001: val_loss improved from inf to 0.05015, saving model to ./kill_log_models\model_t62.hdf5
Epoch 2/20
129/129 [==============================] - 52s 401ms/step - loss: 0.0460 - accuracy: 0.9831 - val_loss: 0.0401 - val_accuracy: 0.9885

Epoch 00002: val_loss improved from 0.05015 to 0.04014, saving model to ./kill_log_models\model_t62.hdf5
Epoch 3/20
129/129 [==============================] - 53s 402ms/step - loss: 0.0270 - accuracy: 0.9931 - val_loss: 0.0601 - val_accuracy: 0.9885

Epoch 00003: val_loss did not improve from 0.04014
Epoch 4/20
129/129 [==============================] - 53s 402ms/step - loss: 0.0294 - accuracy: 0.9914 - val_loss: 0.0342 - val_accuracy: 0.9885

Epoch 00004: val_loss improved from 0.04014 to 0.03422, saving model to ./kill_log_models\model_t62.hdf5
Epoch 5/20
129/129 [==============================] -

In [33]:
# 평가 데이터 결합

In [36]:
# 파인튜닝전 모델 평가

NameError: name 'func_model_t6' is not defined

In [ ]:
# 파인튜닝후 모델 평가

In [38]:
# 가장 좋은 모델 로드하기

In [39]:
# 가장 좋은 모델 평가

24/24 [==============================] - 3s 110ms/step - loss: 0.0570 - accuracy: 0.9840


[0.05701206997036934, 0.9839572310447693]

### 6차 파인튜닝 결과

- 1차, 2차 평가 데이터의 정확도는 98.9%

기존 파인튜닝하기 전의 6차 모델의 1,2차 평가 데이터의 정확도가 97.3%였습니다. 파인튜닝을 통해 약 1.6%의 정확도 상승을 확인할 수 있습니다.  

파인튜닝을 통해서 기존 모델에 대한 고도화까지 진행해보았습니다. 이번에는 전이학습에 사용한 모델을 파라미터 수가 더 많은 VGG16로 변경하여 ResNet의 결과와 비교해보겠습니다.

## 7-1. 7차 학습


### VGG16

- 기존 모델 학습
- ResNet50보다 복잡한 VGG16 사용


### 1차 데이터셋

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 7차 학습 특징


- 전이학습을 통해 기존 모델을 1,2차 데이터셋을 결합하여 전체 데이터에 대한 학습을 진행합니다.
- ROI을 추출한 뒤 Data Augmentation을 통해 데이터셋의 크기를 늘립니다.

In [40]:
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, ReLU, BatchNormalization

In [41]:
# VGG 모델 생성

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [43]:
# 모델 구현

In [45]:
# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장.
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t7.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [46]:
# 모델 학습

Epoch 1/20
129/129 [==============================] - 46s 301ms/step - loss: 0.3783 - accuracy: 0.8280 - val_loss: 0.0951 - val_accuracy: 0.9862

Epoch 00001: val_loss improved from inf to 0.09508, saving model to ./kill_log_models\model_t7.hdf5
Epoch 2/20
129/129 [==============================] - 20s 147ms/step - loss: 0.1337 - accuracy: 0.9718 - val_loss: 0.0565 - val_accuracy: 0.9931

Epoch 00002: val_loss improved from 0.09508 to 0.05647, saving model to ./kill_log_models\model_t7.hdf5
Epoch 3/20
129/129 [==============================] - 20s 148ms/step - loss: 0.1089 - accuracy: 0.9759 - val_loss: 0.0518 - val_accuracy: 0.9931

Epoch 00003: val_loss improved from 0.05647 to 0.05179, saving model to ./kill_log_models\model_t7.hdf5
Epoch 4/20
129/129 [==============================] - 20s 149ms/step - loss: 0.0924 - accuracy: 0.9778 - val_loss: 0.0564 - val_accuracy: 0.9931

Epoch 00004: val_loss did not improve from 0.05179
Epoch 5/20
129/129 [==============================] - 20s

In [47]:
# 모델 평가

24/24 [==============================] - 8s 317ms/step - loss: 0.0936 - accuracy: 0.9693


[0.09361500293016434, 0.9692513346672058]

In [48]:
# 가장 좋은 모델 로드하기

In [49]:
# 가장 좋은 모델 평가

24/24 [==============================] - 3s 130ms/step - loss: 0.0628 - accuracy: 0.9853


[0.06276007741689682, 0.9852941036224365]

### 7차 학습의 결과

- 학습 데이터의 가장 높은 정확도는 99%

- 1,2차 평가 데이터의 정확도는 98.5%
 

전이학습을 위해 사용하였던 기학습 모델을 ResNet 모델에서 VGG 모델로 변경하였습니다. VGG 모델은 ResNet 모델보다 layer의 수는 적으나 파라미터 수가 더 많다는 차이가 존재합니다. 이전 학습에서도 파인튜닝으로 모델을 고도화하여 정확도 향상을 확인하였기 때문에 VGG 모델에 대해서도 파인튜닝을 진행합니다.

## 7-2. 7차 학습에 대한 파인튜닝

평가 데이터의 정확도가 충분히 높기 때문에 파인튜닝을 통해서 현재 모델을 고도화하는 방법을 선택할 수 있습니다.


### VGG16

- 7차 학습 모델 사용


### 1차 데이터셋 

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 7차 파인튜닝 특징


- 전이학습을 위해 가중치 갱신이 불가능하도록 설정한 것을 파인튜닝을 위해 가능하도록 설정합니다.
- 1, 2차의 데이터셋을 결합합니다.
- ROI을 추출한 뒤 Data Augmentation을 통해 데이터셋의 크기를 늘립니다.

In [25]:
# 7차 모델 로드하기

In [26]:
# 7차 모델 학습 가능하도록 변경하기

In [28]:
# val_loss에 따른 빠른 종료 -> 빠른 구현을 위해서 선택적으로 적용할 수 있다.
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장. (모델을 저장하는 것은 ) 
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t72.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [29]:
# 파인튜닝을 위한 학습루프 정의

In [31]:
# 파인튜닝을 위한 추가 학습

Epoch 1/20
129/129 [==============================] - 141s 735ms/step - loss: 0.0410 - accuracy: 0.9880 - val_loss: 0.0313 - val_accuracy: 0.9931

Epoch 00001: val_loss improved from inf to 0.03133, saving model to ./kill_log_models\model_t72.hdf5
Epoch 2/20
129/129 [==============================] - 65s 503ms/step - loss: 0.0303 - accuracy: 0.9907 - val_loss: 0.0298 - val_accuracy: 0.9954

Epoch 00002: val_loss improved from 0.03133 to 0.02981, saving model to ./kill_log_models\model_t72.hdf5
Epoch 3/20
129/129 [==============================] - 66s 504ms/step - loss: 0.0160 - accuracy: 0.9947 - val_loss: 0.0276 - val_accuracy: 0.9954

Epoch 00003: val_loss improved from 0.02981 to 0.02761, saving model to ./kill_log_models\model_t72.hdf5
Epoch 4/20
129/129 [==============================] - 66s 505ms/step - loss: 0.0139 - accuracy: 0.9956 - val_loss: 0.0304 - val_accuracy: 0.9954

Epoch 00004: val_loss did not improve from 0.02761
Epoch 5/20
129/129 [==============================] -

In [34]:
# 파인튜닝 모델 평가

24/24 [==============================] - 93s 4s/step - loss: 0.0407 - accuracy: 0.9920


[0.0407470278441906, 0.9919785857200623]

In [35]:
# 가장 좋은 모델 로드하기

In [36]:
# 가장 좋은 모델 평가

24/24 [==============================] - 38s 2s/step - loss: 0.0450 - accuracy: 0.9906


[0.045024774968624115, 0.990641713142395]

In [37]:
# 지금까지 best 모델 저장하기
func_model_t72.save('./kill_log_models/best_model.hdf5')

### 7차 파인튜닝의 결과

- 학습 데이터의 가장 높은 정확도는 99%

- 1,2차 평가 데이터의 정확도는 100%
 
VGG 모델을 사용하여 구현한 모델의 정확도가 ResNet 모델을 사용하여 구현한 모델이 성능이 더 좋은 것을 확인할 수 있었습니다. 하지만 평가 데이터셋이 1,2차를 합치더라도 매우 적기 때문에 현재 모델의 성능을 판단하기 어렵습니다. 그러므로 새로 수집되는 리그오브레전드 이미지에 대해 모델을 적용해봄으로써 현재 모델의 성능을 판단해보겠습니다.